In [ ]:
!pip install datasets
!pip install transformers
!pip install evaluate
#had to re-run this one to check whether my model was taking into account gold labels or not (it is not doing that which is how it should be)
#original accuracy was 84.6% as listed on my presentation
#about 5 reruns had accuracy in the 83s and 84s lower than the original accuracy
#finally got it >= the original accuracy (right now it is 85.3%) and confirmed that it is not taking into account gold standard/true labels
#another metric used in the paper, the testing time also improved from 5.9s to 4.7s (along with the speedup(2.12x -> 2.66x) as it was derived from testing time)
#I have not changed anything in my presentation because it was what is was 
#Instead, I just documented everything here
#Another thing, this is just what I have oberserved, Colab seems to always have better metrics when run on Chrome than when run on Safari
#And here is a major but true assumption that I have made: If this model does not take into account gold labels during prediction/evauation, then none of my other models for this final project do

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 14.3 MB/s 
     |████████████████████████████████| 182 kB 90.7 MB/s 
     |████████████████████████████████| 115 kB 87.8 MB/s 
     |████████████████████████████████| 212 kB 90.4 MB/s 
     |████████████████████████████████| 127 kB 84.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 15.0 MB/s 
     |████████████████████████████████| 7.6 MB 66.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.6 MB/s 


In [ ]:
from datasets import load_dataset
#For CBT, the authors chose NE and CN only

datasetRT = load_dataset("rotten_tomatoes")
# datasetCBTNE = load_dataset("cbt", "NE")
# datasetCBTCN = load_dataset("cbt", "CN")
# # you can use any of the following config names as a second argument:
# "CN", "NE", "P", "V", 
# "raw"
# datasetAG = load_dataset("ag_news")
# datasetIMDB = load_dataset("imdb")


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# print("AGN", datasetAG)
# print("IMDB", datasetIMDB)
# print("CBT", datasetCBTNE)
# print("CBT", datasetCBTCN)
print("RT", datasetRT)

RT DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})


In [ ]:
from datasets import Dataset, load_metric, ClassLabel
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import os
import json
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
	logits,labels = eval_pred
	predictions = np.argmax(logits, axis=-1)
	result = metric.compute(predictions=predictions, references=labels)
	return result

# #IMDB sentiment analysis - We randomly set aside about 15% of training data as validation set.
# datasetIMDB_train = datasetIMDB["train"].train_test_split(test_size=0.15)
# # datasetIMDB_validate = datasetIMDB_train["test"]
# datasetIMDB_test = datasetIMDB["test"]
# print(datasetIMDB_train, datasetIMDB_test)

# #CBT sentence level auto QA - 
# datasetCBTNE_train = datasetCBTNE["train"]
# datasetCBTNE_test = datasetCBTNE["test"]
# datasetCBTCN_train = datasetCBTCN["train"]
# datasetCBTCN_test = datasetCBTCN["test"]
# #AG char level news article classification - There are 30,000 training and 1,900 testing examples for each class respectively, where 15% of training data is set aside as validation.
# datasetAG_train = datasetAG["train"].train_test_split(test_size=0.15)
# # datasetAG_validate = datasetAG_train["test"]
# datasetAG_test = datasetAG["test"]
# print(datasetAG_train, datasetAG_test)

In [ ]:
print("building tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
	token = tokenizer(examples["text"], padding="max_length", truncation=True)
	return token

tokenized_datasetRT = datasetRT.map(tokenize_function, batched=True)
print(tokenized_datasetRT)

building tokenizer...


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1066
    })
})


In [ ]:
os.environ["WANDB_DISABLED"] = "true" #Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length")
modelRT = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 2)
modelRT.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/ColabNotebooks/model out/",
	optim = "adamw_torch", 
	evaluation_strategy="steps", 
	#num_train_epochs = 10,
	metric_for_best_model = "accuracy",
	greater_is_better = True,
  warmup_steps = 5000,
	save_steps=5000,
	max_steps = 25000,
	eval_steps = 5000,
  # warmup_steps = 2000,
	# save_steps=2000,
	# max_steps = 10000,
	# eval_steps = 2000,
	per_device_train_batch_size=8, 
	per_device_eval_batch_size=8,
	weight_decay=0.01,
	# learning_rate = 2e-5,#, #defaults to 5e-5
	learning_rate = 5e-6,
	load_best_model_at_end = True,
	# lr_scheduler_type = "cosine"
	# report_to="wandb"
	)

trainer = Trainer(
		model=modelRT,
		args=training_args,
		train_dataset=tokenized_datasetRT["train"],
    eval_dataset=tokenized_datasetRT["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
		compute_metrics=compute_metrics
)
trainer.train()

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

Step,Training Loss,Validation Loss,Accuracy
5000,0.272400,0.555682,0.839587
10000,0.073900,0.836465,0.841463
15000,0.044100,1.068381,0.848968
20000,0.005900,1.328226,0.848968
25000,0.005100,1.401363,0.854597


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1066
  Batch size = 8
Saving model checkpoint to /content/drive/My Drive/ColabNotebooks/model out/checkpoint-5000
Configuration saved in /content/drive/My Drive/ColabNotebooks/model out/checkpoint-5000/config.json
Model weights saved in /content/drive/My Drive/ColabNotebooks/model out/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/ColabNotebooks/model out/checkpoint-5000/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/ColabNotebooks/model out/checkpoint-5000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forwar

TrainOutput(global_step=25000, training_loss=0.13087847614765166, metrics={'train_runtime': 2916.5487, 'train_samples_per_second': 68.574, 'train_steps_per_second': 8.572, 'total_flos': 2.647519923018547e+16, 'train_loss': 0.13087847614765166, 'epoch': 23.43})

In [ ]:
#get predictions on Rotten Tomatoes Test set for task 2

#Zaid told me about how I could check if my model is using gold labels for prediction (It is not, I think)
#Random number does NOT mean a different number every time. Random means something that can not be predicted logically.
import copy
from numpy import random



real_test_set = tokenized_datasetRT['test']
fake_test_set = copy.deepcopy(tokenized_datasetRT['test'])
for l in fake_test_set['label']:
  l = random.randint(2)

# Go into the fake_test_set and set labels to random values

print(tokenized_datasetRT['test'])
prediction = trainer.predict(tokenized_datasetRT['test'])
prediction1 = trainer.predict(fake_test_set)
print(prediction)
print(prediction1)

#accuracy for both is same

# # For each prediction, create the label with argmax
# test_predictions_argmax = np.argmax(prediction[0], axis=1)
# # Retrieve reference labels from test set
# test_references = np.array(datasetIMDB_test["label"])
# # Compute accuracy
# metric.compute(predictions=test_predictions_argmax, references=test_references)
# # {'accuracy': 0.86028} same as trainer output when given labels

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1066
})


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


PredictionOutput(predictions=array([[-4.221517 ,  5.301756 ],
       [-4.7067456,  5.8852687],
       [ 3.035723 , -3.1810017],
       ...,
       [ 6.007427 , -6.605852 ],
       [ 5.797522 , -6.464785 ],
       [ 4.8048763, -5.3491673]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 1.3869810104370117, 'test_accuracy': 0.8527204502814258, 'test_runtime': 4.7095, 'test_samples_per_second': 226.351, 'test_steps_per_second': 28.453})
PredictionOutput(predictions=array([[-4.221517 ,  5.301756 ],
       [-4.7067456,  5.8852687],
       [ 3.035723 , -3.1810017],
       ...,
       [ 6.007427 , -6.605852 ],
       [ 5.797522 , -6.464785 ],
       [ 4.8048763, -5.3491673]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 1.3869810104370117, 'test_accuracy': 0.8527204502814258, 'test_runtime': 4.713, 'test_samples_per_second': 226.182, 'test_steps_per_second': 28.432})
